In [7]:
import nltk
import pandas as pd
import sys
import os

from data_loader import DataLoader

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import re

from collections import defaultdict
import string

In [2]:
loader = DataLoader(DataLoader.data_path2)
table = loader.load_table()
table.head()

C:\Users\lijia\Desktop\MSAI\Sem A\AI6122\Assignment1\Hearts-of-Iron\data_loader.py:13: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,4,NaN,True,"03 18, 2016",A1S2L9211DXHL3,B017PDBZ1M,{'Format:': ' MP3 Music'},Eric H.,Delightful!,Four Stars,1458259200,NaN
1,5,NaN,True,"10 30, 2014",AW6WK8Q7UHAE6,B00KF2KMME,{'Format:': ' MP3 Music'},Melinda Beckham,Great,Five Stars,1414627200,NaN
2,5,NaN,True,"07 28, 2015",A5FYUZ5861YSJ,B001D7JRJO,{'Format:': ' MP3 Music'},Charlene Morissaint,Love it,Five Stars,1438041600,NaN
3,5,NaN,True,"05 19, 2014",A2378Y827M4QLC,B0011ZP544,{'Format:': ' MP3 Music'},JAS,What more can I say about this song that you d...,So Strong and Powerful!,1400457600,NaN
4,5,NaN,True,"12 15, 2017",A33FCZANJ9BWS7,B001KRW5UK,{'Format:': ' MP3 Music'},bible lover,REMEMBER THE GOOD THINGS YOUR MOTHERS USED TO ...,NICE THROWBACK THAT NEEDS TO BE THROWN BACK IN,1513296000,NaN


In [3]:
#Data Preprocess
#Get stopwords
nltk_stopwords = stopwords.words('english')
nltk_stopwords.append('\n')
spacy_stopwords = list(STOP_WORDS)
merged_sw = nltk_stopwords + list(set(spacy_stopwords) - set(nltk_stopwords))

print("nltk: ", len(nltk_stopwords))
print("spacy: ", len(spacy_stopwords))
print("merge: ", len(merged_sw))

nltk:  180
spacy:  326
merge:  383


In [4]:
#Create the review text dictionary
reviewText = {}
for i in range(200):
    if table['asin'][i] in reviewText:
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
    else:
        reviewText[table['asin'][i]] = []
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
print(reviewText)

{'B017PDBZ1M': ['Delightful!'], 'B00KF2KMME': ['Great'], 'B001D7JRJO': ['Love it'], 'B0011ZP544': ["What more can I say about this song that you don't arleady know, feel, and relealize when you listen to the words.", 'This song sings to my soul!', 'Just beautiful!'], 'B001KRW5UK': ['REMEMBER THE GOOD THINGS YOUR MOTHERS USED TO SAY THAT ALWAYS MADE YOUR DAY!', 'SELF EXPLANATORY.', 'NICE  THROWBACK THAT NEEDS TO BE THROWN BACK IN!', 'WILL BE IN THE LIBRARY FOR MANY MANY YEARS TO COME!'], 'B000QOMANK': ['Another Master of entertainment we shall never ever forget.', 'Inspiring and delightful.', 'Makes you smile every time you hear him and ashamed of how you concern yourself with your own meager problems when you remember how he overcame so much at that time.'], 'B00123NB5K': ['This song sounds great on my Kindle and Roku!', 'I am so glad that I purchased it!', 'It was definitely a great find!'], 'B0011Z4YB4': ['Love the song, love the movie Purple Rain, love Prince!'], 'B00NYII6F6': ['Nic

In [5]:
def data_preprocess(original_review, stop_words):
    # to lowercase
    review = original_review.lower()
    # remove punctuation
    review = re.sub(r'[^\w\s]', ' ', review).strip()
    lemmatizer = WordNetLemmatizer()
    review_words = review.split(' ')
    review_words_tag = nltk.pos_tag(review_words)
    processed_review = ""
    # remove stop words&lemma
    for word, tag in review_words_tag:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma_word = word
        else:
            lemma_word = lemmatizer.lemmatize(word, wntag)
        if lemma_word not in stop_words:
            processed_review = processed_review + lemma_word + " "

    processed_review = processed_review.strip()
    return processed_review



In [6]:
processed_reviewText = {}
for r in reviewText:
    processed_reviewText[r] = []
    for t in reviewText[r]:
        p = data_preprocess(t, merged_sw)
        processed_reviewText[r].append(p)
print(processed_reviewText)

{'B017PDBZ1M': ['delightful'], 'B00KF2KMME': ['great'], 'B001D7JRJO': ['love'], 'B0011ZP544': ['song arleady know  feel  relealize listen word', 'song sing soul', 'beautiful'], 'B001KRW5UK': ['remember good thing mother use day', 'self explanatory', 'nice  throwback need throw', 'library year come'], 'B000QOMANK': ['master entertainment shall forget', 'inspiring delightful', 'smile time hear ashamed concern meager problem remember overcome time'], 'B00123NB5K': ['song sound great kindle roku', 'glad purchase', 'definitely great find'], 'B0011Z4YB4': ['love song  love movie purple rain  love prince'], 'B00NYII6F6': ['nice song'], 'B001BW1TRE': ['80 fan'], 'B001BZLFDE': ['good music'], 'B000TDB4R6': ['granddaughter love song  4  mini ipod sing time'], 'B0043ZDFEQ': ['cool'], 'B0011Z76XW': ['great instrumental  flash past'], 'B00GM5Y67A': ['beautiful song', 'listen'], 'B00122PH3U': ['fly high   \nnot big fan drug skepticism  live day  song bring memory good solid piece', 'taxi  creative s

In [16]:
# collect frequency

def collect_frequency(processed_review):
    word_frequency = defaultdict(lambda:0)
    for v in processed_review.values():
        for i in v:
            tokens = nltk.word_tokenize(i)
            for token in tokens:
                word_frequency[token] += 1  
    return word_frequency

frequency = collect_frequency(processed_reviewText)

In [18]:
frequency = sorted(frequency.items(), key=lambda item: item[1],reverse=True)
print(frequency)

[('song', 149), ('love', 61), ('great', 53), ('good', 47), ('music', 36), ('like', 33), ('album', 28), ('sound', 27), ('time', 21), ('listen', 18), ('buy', 18), ('think', 16), ('know', 13), ('hear', 13), ('beat', 13), ('soundtrack', 12), ('use', 11), ('bring', 11), ('lyric', 11), ('favorite', 11), ('drum', 11), ('classic', 10), ('guitar', 10), ('band', 10), ('feel', 9), ('nice', 9), ('come', 9), ('find', 9), ('awesome', 9), ('voice', 9), ('download', 9), ('amazon', 9), ('old', 9), ('work', 9), ('new', 9), ('vocal', 9), ('track', 9), ('link', 9), ('sing', 8), ('play', 8), ('star', 8), ('cd', 8), ('best', 8), ('better', 8), ('heavy', 8), ('sepultura', 8), ('max', 8), ('riff', 8), ('beautiful', 7), ('thing', 7), ('fan', 7), ('easy', 7), ('quality', 7), ('sure', 7), ('volume', 7), ('want', 7), ('theme', 7), ('begin', 7), ('soulfly', 7), ('yell', 7), ('need', 6), ('year', 6), ('definitely', 6), ('excellent', 6), ('hold', 6), ('mp3', 6), ('rock', 6), ('fun', 6), ('record', 6), ('product', 6)